In [1]:
from dotenv import load_dotenv
load_dotenv()

import os
project_name = "wanted_2nd_langchain_memory_basic"
os.environ["LANGSMITH_PROJECT"] = project_name

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

model = ChatOpenAI(
    temperature=0.1,
    model="gpt-4.1-mini",
    verbose=True
)

In [3]:
from typing import Dict
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

In [15]:
# 1. 프롬프트 자리에 히스토리 파트 확보
system_prompt = """
당신은 사람의 심리상태를 분석하는 전문가 입니다.
밑에 [FORMAT]에 맞게 답변해줘

[FORMAT]
1. 말투를 파악하여 어떠한 감정을 가지고 있는지 파악
2. 심리상태에 따른 solution 제시
3. 감정에 맞는 말투를 사용
4. 예시) 감정상태에 따른 점수:
        행복: ""
        불안: ""
        행복: ""
        슬플: ""
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    MessagesPlaceholder(variable_name="history"),
    ("user", "{question}")
])

chain = prompt | model | StrOutputParser()

In [16]:
# 2. 세션별 저장소 구성하기
stores : Dict[str, InMemoryChatMessageHistory] = {}

K = 6
def get_stores(session_id : str):
    if session_id not in stores:    # 아직 대화를 한번도 나눈적이 없는 경우
        stores[session_id] = InMemoryChatMessageHistory()

    hist = stores.setdefault(session_id, InMemoryChatMessageHistory())

    if len(hist.messages) > K:
        hist.messages[:] = hist.messages[-K:]

    return hist

In [17]:
# 3. 히스토리와 연결
with_history = RunnableWithMessageHistory(
    chain, 
    get_stores,
    input_messages_key="question",
    history_messages_key="history"
)

In [18]:
config = {"configurable" : {"session_id" : "user-123"}}
result = with_history.invoke({"question":"오늘은 날씨가 좋아서 기분이 좋군요"}, config=config)
print(result)
stores["user-123"].messages

1. 말투를 파악하여 어떠한 감정을 가지고 있는지 파악  
   - 말투가 밝고 긍정적이며, 기분이 좋은 상태로 보입니다. 날씨가 좋아서 기분이 좋아졌다는 표현에서 행복감과 만족감을 느끼고 있음을 알 수 있습니다.

2. 심리상태에 따른 solution 제시  
   - 현재 긍정적인 감정 상태를 유지하기 위해, 오늘 같은 좋은 날씨를 활용해 야외 활동이나 산책을 해보는 것을 추천합니다. 자연과의 접촉은 기분을 더욱 좋게 하고 스트레스 해소에도 도움이 됩니다.

3. 감정에 맞는 말투를 사용  
   - "오늘처럼 좋은 날씨에 기분이 좋아지셨다니 정말 다행이에요! 이런 좋은 기분을 계속 유지하시면서 즐거운 하루 보내시길 바랄게요."

4. 예시) 감정상태에 따른 점수:  
   - 행복: 90  
   - 불안: 10  
   - 슬픔: 5


[HumanMessage(content='오늘은 날씨가 좋아서 기분이 좋군요', additional_kwargs={}, response_metadata={}),
 AIMessage(content='1. 말투를 파악하여 어떠한 감정을 가지고 있는지 파악  \n   - 말투가 밝고 긍정적이며, 기분이 좋은 상태로 보입니다. 날씨가 좋아서 기분이 좋아졌다는 표현에서 행복감과 만족감을 느끼고 있음을 알 수 있습니다.\n\n2. 심리상태에 따른 solution 제시  \n   - 현재 긍정적인 감정 상태를 유지하기 위해, 오늘 같은 좋은 날씨를 활용해 야외 활동이나 산책을 해보는 것을 추천합니다. 자연과의 접촉은 기분을 더욱 좋게 하고 스트레스 해소에도 도움이 됩니다.\n\n3. 감정에 맞는 말투를 사용  \n   - "오늘처럼 좋은 날씨에 기분이 좋아지셨다니 정말 다행이에요! 이런 좋은 기분을 계속 유지하시면서 즐거운 하루 보내시길 바랄게요."\n\n4. 예시) 감정상태에 따른 점수:  \n   - 행복: 90  \n   - 불안: 10  \n   - 슬픔: 5', additional_kwargs={}, response_metadata={})]

In [21]:
stores["user-123"].messages

[HumanMessage(content='\n저희는 처음에 YOLO11m모델 Object ditection을 활용한 태양광 패널 이상 객체 탐지를 위해 roboflow 사이트에서 3개의 태양광 패널 데이터셋을 확보하여 데이터가 부족한 클래스는 삭제하고 비슷한 클래스끼리 합쳐주는 방법으로 10,130개의 데이터셋을 확보하였고 확보한 데이터는 클래스별로 바운딩박스가 되어 있었으며 이 데이터셋을 가지고 전이학습을 진행하였고 생각보다 성능이 잘 나오지 않아서 직접 수작업으로 데이터셋을 바운딩박스 작업을 했습니다. 아쉬웠던 점은 클래스별로 모양과 위치가 가지각색인데 사각형 박스로만 객체를 표시 할 수 있기 때문에 섬세한 작업을 할 수 없었습니다. 5천개 정도를 직접 바운딩박스 작업을 한 뒤 학습을 진행하였지만 오히려 학습결과가 안 좋게 나왔습니다. 그래서 저희가 생각한 대안은 YOLO Segmentaion을 활용하여 태양광 패널의 문제가 되는 부분을 클래스별 객체의 외곽선을 따라 픽셀 단위로 정확히 분리한 새로운 3,343개의 외부충격 데이터셋을 가지고 Base Fine-tuning을 진행하였습니다. 왜냐하면 사용하려는 yolo11m-seg.pt 모델은 태양광 패널에 대한 지식이 없기 때문에 새로운 지식을 학습 시켜서 성능을 높이고 이전에 학습 된 best.pt는 검증 기반으로 가장 성능이 좋았던 가중치를 불러와서 학습을 시키면 학습 결과에 미치는 영향은 "빠른 수렴", "데이터가 적을 때의 일반화 향상", "도메인 특화", "학습 실패 극복/ 재현성", 연속 연구/모델 개선 등의 효과가 있다는 것을 알게 되었고 실제 학습결과와 테스트를 해보았을 때 좋은 결과가 나왔습니다. 추가로 태양광 패널 내부충격 데이터셋을 수집하였고 내부충격은 핫스팟이라는 열화상 카메라로 촬영한 데이터이며 특정 지점의 온도가 주변보다 비정상적으로 높은 부분을 표시가 되어있는 데이터이며\n', additional_kwargs={}, response_metadata={}),
 AIMessage(co